In [ ]:
# %pip install numpy matplotlib tifffile lxml

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tifffile import TiffFile, imread, imwrite
import json
from pprint import pprint
import xml.dom.minidom
import sciebo

In [ ]:
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/3Uf2gScrvuTPQhB', 'data/steinmetz_2017-01-08_Muller.nc')


## Reading TIFF File 

TIFF, which stands for Tagged Image File Format, is a flexible and adaptable file format for storing images and data within a single file. It can be used to store multiple frames as `pages`.

**Example** Read `data_endoscope.tif`. How many frames does it have?

In [ ]:
frames = imread('../data/data_endoscope.tif')
frames.shape

Read `Sue_2x_3000_40_-46.tif`. How many frames does this data have?

In [ ]:
frames = imread('../data/Sue_2x_3000_40_-46.tif')
frames.shape

Read `Sue_2x_3000_40_-46.tif`. How long does it take?

Hint: Put `%%timeit` in the beginning of the cell

In [ ]:
%%timeit
frames = imread('../data/Sue_2x_3000_40_-46.tif')

Read `Sue_2x_3000_40_-46.tif`. How long does it take to access the last frame?

Hint: Put `%%timeit` in the beginning of the cell

In [ ]:
%%timeit
frames = imread('../data/Sue_2x_3000_40_-46.tif')
frames[-1]

**Example** Let's see the plots. Plot the first frame

In [ ]:
plt.imshow(frames[0], cmap='gray')

Plot the last frame

In [ ]:
plt.imshow(frames[-1], cmap='gray')

**Example** Read only the first 3 frames from `Sue_2x_3000_40_-46.tif`

In [ ]:
frames = imread('../data/Sue_2x_3000_40_-46.tif', key=(0, 1, 2))
frames.shape

Read only 1, 3, 5, 7, 9th frame from `Sue_2x_3000_40_-46.tif`

In [ ]:
frames = imread('../data/Sue_2x_3000_40_-46.tif', key=(0, 2, 4, 6, 8))
frames.shape

Read only first 100 frames from `Sue_2x_3000_40_-46.tif`

Hint: use range(0, 100)

In [ ]:
frames = imread('../data/Sue_2x_3000_40_-46.tif', key=range(0, 100))
frames.shape

**Example** Read `data_endoscope.tif` with `TiffFile`?

In [ ]:
f = TiffFile('../data/data_endoscope.tif')
frames = np.array([page.asarray() for page in f.pages])

Read `Sue_2x_3000_40_-46.tif` with TiffFile

In [ ]:
f = TiffFile('../data/Sue_2x_3000_40_-46.tif')
frames = np.array([page.asarray() for page in f.pages])

How long does it take to read `Sue_2x_3000_40_-46.tif` with TiffFile?

In [ ]:
%%timeit
f = TiffFile('../data/Sue_2x_3000_40_-46.tif')
frames = np.array([page.asarray() for page in f.pages])

How long does it take to access only the last frame with TiffFile?

In [ ]:
%%timeit
f = TiffFile("../data/data_endoscope.tif") 
f.pages[-1].asarray()

## Metadata handling and TIFF file writing 

In [ ]:
f = TiffFile('../data/data_endoscope.tif')
print(f)

In [ ]:
f = TiffFile('../data/Sue_2x_3000_40_-46.tif')
print(f)

In [ ]:
f = TiffFile('../data/data_endoscope.tif')
print(f.pages[0].tags)

In [ ]:
f = TiffFile('../data/data_endoscope.tif')
print(f.pages[499].tags)

In [ ]:
f = TiffFile('../data/data_endoscope.tif')
print(f.pages[999].tags)

In [ ]:
f = TiffFile('../data/data_endoscope.tif')
print(f.pages[999].tags['ImageWidth'].value)

In [ ]:
f = TiffFile('../data/data_endoscope.tif')
print(f.pages[999].tags['PhotometricInterpretation'].value)

In [ ]:
f = TiffFile('../data/data_endoscope.tif')
print(f.pages[999].tags['ResolutionUnit'].value)

In [ ]:
frame1 = np.random.rand(100, 100)  # Frame 1
frame2 = np.random.rand(100, 100)  # Frame 2
frame3 = np.random.rand(100, 100)  # Frame 3

In [ ]:
tiff_path = 'output/output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], photometric='minisblack')

f = TiffFile('output/output_file.tiff')
print(f)

In [ ]:
tiff_path = 'output/output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], photometric='minisblack')

f = TiffFile('output/output_file.tiff')
print(f.pages[-1].tags)

In [ ]:
tiff_path = 'output/output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], photometric='rgb')

f = TiffFile('output/output_file.tiff')
print(f.pages[-1].tags['PhotometricInterpretation'].value)

In [ ]:
metadata = {
    'Experiment': 'Calcium Imaging',
}
metadata_json = json.dumps(metadata)
tiff_path = 'output/output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], description=metadata_json)

f = TiffFile('output/output_file.tiff')
print(f.pages[0].tags['ImageDescription'].value)

In [ ]:
metadata = {
    'Experiment': 'Calcium Imaging',
    'Date': '2024-02-25'
}
metadata_json = json.dumps(metadata)
tiff_path = 'output/output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], description=metadata_json)

f = TiffFile('output/output_file.tiff')
print(f.pages[0].tags['ImageDescription'].value)

In [ ]:
metadata = {
    'Experiment': 'Calcium Imaging',
    'Date': '2024-02-25',
    'Researcher': 'Sangee'
}
metadata_json = json.dumps(metadata)
tiff_path = 'output/output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], description=metadata_json)

f = TiffFile('output/output_file.tiff')
print(f.pages[0].tags['ImageDescription'].value)

## OME data model and structure


**Example** Read and print OME metadata from output_file.tiff. Is there any?

In [ ]:
f = TiffFile('output/output_file.tiff')
ome_metadata = f.ome_metadata
print(ome_metadata)

Read and print ome metadata of `output.ome.tiff`. Is it readable?

In [ ]:
f = TiffFile('output/output.ome.tiff')
ome_metadata = f.ome_metadata
print(ome_metadata)

Let's make it more readable. Instead of `print` use `pprint` which means `pretty print`

In [ ]:
f = TiffFile('output/output.ome.tiff')
ome_metadata = f.ome_metadata
pprint(ome_metadata)

Better. The format of the output is XML. XML, which stands for eXtensible Markup Language, is a markup language that defines a set of rules for encoding documents in a format that is both human-readable and machine-readable. 

**Example** Read ome metadata of output.ome.tiff and use inbuilt library `xml` to parse the string format to pretty xml. 

In [ ]:
f = TiffFile('output/output.ome.tiff')
ome_metadata = f.ome_metadata
dom = xml.dom.minidom.parseString(ome_metadata) 
pprint(dom.toprettyxml())

Read ome metadata of multifile-Z1.ome.tiff and use inbuilt library `xml` to parse the string format to pretty xml 

In [ ]:
f = TiffFile('../data/multifile-Z1.ome.tiff')
ome_metadata = f.ome_metadata
dom = xml.dom.minidom.parseString(ome_metadata) 
pprint(dom.toprettyxml())

https://downloads.openmicroscopy.org/images/OME-TIFF/2016-06/MitoCheck/

Let's use `00001_01.ome.tiff` for the rest of the exercises. Read its metadata and pretty print in pretty xml.

Looking through its data, what are the X, Y, and temporal dimensions?

In [ ]:
f = TiffFile('output/00001_01.ome.tiff')
ome_metadata = f.ome_metadata
dom = xml.dom.minidom.parseString(ome_metadata) 
pprint(dom.toprettyxml())

Confirm the dimensions by checking shape of the image data

In [ ]:
f = TiffFile('output/00001_01.ome.tiff')
image_data = f.asarray()
image_data.shape

**Example** It's been a while since we visualized something! Let's see how the first frame looks like? What kind of data is this?

In [ ]:
f = TiffFile('output/00001_01.ome.tiff')
image_data = f.asarray()
plt.imshow(image_data[0]);

Let's see how the second frame looks like.

In [ ]:
f = TiffFile('output/00001_01.ome.tiff')
image_data = f.asarray()
plt.imshow(image_data[1]);

Let's see how the last frame looks like.

In [ ]:
f = TiffFile('output/00001_01.ome.tiff')
image_data = f.asarray()
plt.imshow(image_data[-1]);

In [ ]:
from matplotlib.animation import FuncAnimation

frames = image_data.copy()

fig, ax = plt.subplots()
frame = frames[0]  
im = ax.imshow(frame, animated=True)

def update(frame):
    im.set_array(frame)
    return [im]

ani = FuncAnimation(fig, update, frames=frames, blit=True, interval=50);

from IPython.display import HTML
HTML(ani.to_jshtml())


## Writing OME Data and metadata

In [ ]:
frame1 = np.random.rand(100, 100)  # Frame 1

In [ ]:
metadata = {
    'Experiment': 'Calcium Imaging',
}
tiff_path = 'output/output.ome.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], metadata={'Description': str(metadata)})

f = TiffFile('output/output.ome.tiff')
ome_metadata = f.ome_metadata
pprint(ome_metadata)

Is there any software that can help with this? YES!

Use Bio-Format Importer from Plugins tab in ImageJ to import `00001_01.ome.tiff`. 


It might take a while to laod. Check `open xml`. Feel free to explore other options 